# Initialization

In [1]:
import logging

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

# Загрузка данных

In [3]:
items = pd.read_parquet("items.par")
events = pd.read_parquet("events.par")

# Разбиение с учётом хронологии

Рекомендательные системы на практике работают с учётом хронологии. Поэтому поток событий для тренировки и валидации полезно делить на то, что уже случилось, и что ещё случится. Это позволяет проводить валидацию на тех же пользователях, на которых тренировались, но на их событиях в будущем.

# === Знакомство: "холодный" старт

In [4]:
# зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime("2017-08-01").date()

train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

# количество пользователей в train и test
users_train = events_train["user_id"].drop_duplicates()
users_test = events_test["user_id"].drop_duplicates()
# количество пользователей, которые есть и в train, и в test
common_users  = set(users_train).intersection(set(users_test))

# print(len(users_train), len(users_test), len(common_users))
print(f'Кол-во пользователей в train: {len(users_train)}')
print(f'Кол-во пользователей в test: {len(users_test)}')
print(f'Кол-во пользователей, которые есть и в train, и в test: {len(common_users)}')

Кол-во пользователей в train: 428220
Кол-во пользователей в test: 123223
Кол-во пользователей, которые есть и в train, и в test: 120858


In [5]:
# Второй вариант:


# Разделяем данные по дате "2017-08-01"
split_date = pd.to_datetime("2017-08-01").date()

# Обучающая выборка (данные ДО split_date)
train_data = events[events["started_at"] < split_date]

# Тестовая выборка (данные ПОСЛЕ split_date)
test_data = events[events["started_at"] >= split_date]  # вместо ~ можно явно указать >=

# Уникальные пользователи в train и test
train_users = train_data["user_id"].unique()  # проще, чем drop_duplicates()
test_users = test_data["user_id"].unique()

# Пользователи, которые есть в обоих выборках
common_users = set(train_users) & set(test_users)  # альтернатива intersection

print(
    f"Пользователей в train: {len(train_users)}\n"
    f"Пользователей в test: {len(test_users)}\n"
    f"Общих пользователей: {len(common_users)}"
)

Пользователей в train: 428220
Пользователей в test: 123223
Общих пользователей: 120858


In [6]:
# Идентифицируйте холодных пользователей и оцените их количество.

cold_users = set(users_test) - set(users_train)
print(f"Холодных пользователей: {len(cold_users)}\n")

Холодных пользователей: 2365



# === Знакомство: первые персональные рекомендации

Ключевые моменты:
Фильтрация по дате: Берем только события с 2015 года

Группировка и агрегация:
    - users - количество уникальных пользователей для каждой книги
    - avg_rating - средний рейтинг книги

Нормализация:
Приводим метрики к единому масштабу (0-1) для корректного объединения

Popularity Score:
Комбинированная метрика = нормированное кол-во пользователей × нормированный рейтинг

Финальный отбор:
- Сначала фильтруем по рейтингу (>= 4)
- Затем берем топ-100 по popularity_score

Этот подход обеспечивает рекомендации, которые одновременно:
- Популярны (много пользователей взаимодействовали)
- Качественны (высокий средний рейтинг)
- Актуальны (учитываются только свежие данные с 2015 года)

In [7]:
from sklearn.preprocessing import MinMaxScaler

top_pop_start_date = pd.to_datetime("2015-01-01").date()

item_popularity = events_train \
    .query("started_at >= @top_pop_start_date") \
    .groupby(["item_id"]).agg(users=("user_id", "nunique"), avg_rating=("rating", "mean")).reset_index()

# нормализация пользователей и среднего рейтинга, требуется для их приведения к одному масштабу
scaler = MinMaxScaler()
item_popularity[["users_norm", "avg_rating_norm"]] = scaler.fit_transform(
    item_popularity[["users", "avg_rating"]]
)

# вычисляем popularity_score, как скор популярности со штрафом за низкий рейтинг
item_popularity["popularity_score"] = (
    item_popularity["users_norm"] * item_popularity["avg_rating_norm"]
)

# сортируем по убыванию popularity_score
item_popularity = item_popularity.sort_values(by = 'popularity_score', ascending = False)

# выбираем первые 100 айтемов со средней оценкой avg_rating не меньше 4
top_k_pop_items = item_popularity[item_popularity['avg_rating'] > 4].head(100)

In [8]:
top_k_pop_items

,item_id,users,avg_rating,users_norm,avg_rating_norm,popularity_score
32387,18007564,20207,4.321275,0.496596,0.830319,0.412333
32623,18143977,19462,4.290669,0.478287,0.822667,0.393471
2,3,15139,4.706057,0.372042,0.926514,0.344702
30695,16096824,16770,4.301014,0.412126,0.825253,0.340108
1916,15881,13043,4.632447,0.320529,0.908112,0.291076
...,...,...,...,...,...,...
24837,8490112,4792,4.080968,0.117747,0.770242,0.090694
33611,18966819,4361,4.374914,0.107154,0.843729,0.090409
378,3636,4667,4.098564,0.114675,0.774641,0.088832
32835,18293427,4674,4.092640,0.114847,0.773160,0.088795


In [9]:
# Сколько пользователей оценило книгу, попавшую на первое место в top_k_pop_items?

if not top_k_pop_items.empty:
    top_item_users = top_k_pop_items.iloc[0]["users"]
    print(f"Книга №1 в топ-100 была оценена {top_item_users} пользователями.")
else:
    print("Топ-100 пуст — возможно, нет книг с рейтингом ≥ 4.")

Книга №1 в топ-100 была оценена 20207.0 пользователями.


Добавив информацию о книгах, можно просмотреть, какие попали в топ.

In [10]:
# добавляем информацию о книгах
top_k_pop_items = top_k_pop_items.merge(
    items.set_index("item_id")[["author", "title", "genre_and_votes", "publication_year"]], on="item_id")

with pd.option_context('display.max_rows', 100):
    display(top_k_pop_items[["item_id", "author", "title", "publication_year", "users", "avg_rating", "popularity_score", "genre_and_votes"]]) 

,item_id,author,title,publication_year,users,avg_rating,popularity_score,genre_and_votes
0,18007564,Andy Weir,The Martian,2014,20207,4.321275,0.412333,"{'Science Fiction': 11966, 'Fiction': 8430}"
1,18143977,Anthony Doerr,All the Light We Cannot See,2014,19462,4.290669,0.393471,"{'Historical-Historical Fiction': 13679, 'Fict..."
2,3,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,1997,15139,4.706057,0.344702,"{'Fantasy': 59818, 'Fiction': 17918, 'Young Ad..."
3,16096824,Sarah J. Maas,A Court of Thorns and Roses (A Court of Thorns...,2015,16770,4.301014,0.340108,"{'Fantasy': 14326, 'Young Adult': 4662, 'Roman..."
4,15881,"J.K. Rowling, Mary GrandPré",Harry Potter and the Chamber of Secrets (Harry...,1999,13043,4.632447,0.291076,"{'Fantasy': 50130, 'Young Adult': 15202, 'Fict..."
5,38447,Margaret Atwood,The Handmaid's Tale,1998,14611,4.232770,0.290194,"{'Fiction': 15424, 'Classics': 9937, 'Science ..."
6,11235712,Marissa Meyer,"Cinder (The Lunar Chronicles, #1)",2012,14348,4.179189,0.280247,"{'Young Adult': 10539, 'Fantasy': 9237, 'Scien..."
7,17927395,Sarah J. Maas,A Court of Mist and Fury (A Court of Thorns an...,2016,12177,4.730640,0.279094,"{'Fantasy': 10186, 'Romance': 3346, 'Young Adu..."
8,5,"J.K. Rowling, Mary GrandPré",Harry Potter and the Prisoner of Azkaban (Harr...,2004,11890,4.770143,0.275401,"{'Fantasy': 49784, 'Young Adult': 15393, 'Fict..."
9,13206900,Marissa Meyer,"Winter (The Lunar Chronicles, #4)",2015,12291,4.534293,0.266881,"{'Fantasy': 4835, 'Young Adult': 4672, 'Scienc..."


In [11]:
# # Объедините event_test с top_k_pop_items, чтобы получить рекомендации для холодных пользователей
cold_users_events_with_recs = \
    events_test[events_test["user_id"].isin(cold_users)] \
    .merge(top_k_pop_items, on="item_id", how="left")

cold_user_items_no_avg_rating_idx = cold_users_events_with_recs["avg_rating"].isnull()
cold_user_recs = cold_users_events_with_recs[~cold_user_items_no_avg_rating_idx] \
    [["user_id", "item_id", "rating", "avg_rating"]] 

# Проверьте количество строк, чтобы убедиться, что оно осталось прежним
original_row_count = len(events_test[events_test["user_id"].isin(cold_users)])
current_row_count = len(cold_users_events_with_recs)

# Проверяем, одинаково ли количество строк
original_row_count, current_row_count

(9672, 9672)

In [12]:
# 1. Объединяем events_test с топ-100 книгами (только для холодных пользователей)
cold_users_events_with_recs = (
    events_test[events_test["user_id"].isin(cold_users)]
    .merge(top_k_pop_items, on="item_id", how="left")
)

# 2. Фильтруем только книги из топ-100 (avg_rating не NaN)
cold_user_recs = cold_users_events_with_recs[
    ~cold_users_events_with_recs["avg_rating"].isnull()
][["user_id", "item_id", "rating", "avg_rating"]]

# 3. Проверяем, что merge не изменил количество строк
original_row_count = len(events_test[events_test["user_id"].isin(cold_users)])
current_row_count = len(cold_users_events_with_recs)

# Вывод результатов
print(
    f"Исходное количество записей холодных пользователей: {original_row_count}\n"
    f"Количество записей после merge: {current_row_count}\n"
    f"Осталось записей после фильтрации (топ-100 книг): {len(cold_user_recs)}"
)

Исходное количество записей холодных пользователей: 9672
Количество записей после merge: 9672
Осталось записей после фильтрации (топ-100 книг): 1912


In [14]:
# Проверка равенства (должно быть True)
assert original_row_count == current_row_count, "Количество строк изменилось после merge!"
print("✅ Проверка пройдена: merge не изменил количество строк.")

✅ Проверка пройдена: merge не изменил количество строк.


In [15]:
# Для какой доли событий «холодных» пользователей в events_test рекомендации в top_k_pop_items совпали по книгам? Округлите ответ до сотых.

# 1. События холодных пользователей
cold_events = events_test[events_test["user_id"].isin(cold_users)]

# 2. Совпадения с топ-100 книгами
matched_events = cold_events[cold_events["item_id"].isin(top_k_pop_items["item_id"])]

# 3. Доля совпадений (с округлением)
match_ratio = len(matched_events) / len(cold_events)
match_ratio_rounded = round(match_ratio, 2)

print(f"Доля событий холодных пользователей с книгами из топ-100: {match_ratio_rounded:.2f}")

Доля событий холодных пользователей с книгами из топ-100: 0.20


In [16]:
# Посчитайте метрики rmse и mae для полученных рекомендаций.

from sklearn.metrics import mean_squared_error, mean_absolute_error

# Рассчитываем метрики
rmse = mean_squared_error(cold_user_recs["rating"], cold_user_recs["avg_rating"], squared=False)
mae = mean_absolute_error(cold_user_recs["rating"], cold_user_recs["avg_rating"])

# Красивый вывод с f-строками
print(f"Оценка качества рекомендаций для холодных пользователей:")
print(f"• RMSE (Среднеквадратичная ошибка): {rmse:.2f}")
print(f"• MAE (Средняя абсолютная ошибка): {mae:.2f}")
print(f"\nИнтерпретация:")
print(f"- RMSE = {rmse:.2f} означает, что в среднем ошибка составляет ±{rmse:.2f} балла")
print(f"- MAE = {mae:.2f} показывает среднее абсолютное отклонение предсказаний")

Оценка качества рекомендаций для холодных пользователей:
• RMSE (Среднеквадратичная ошибка): 0.78
• MAE (Средняя абсолютная ошибка): 0.62

Интерпретация:
- RMSE = 0.78 означает, что в среднем ошибка составляет ±0.78 балла
- MAE = 0.62 показывает среднее абсолютное отклонение предсказаний


In [17]:
# посчитаем покрытие холодных пользователей рекомендациями

cold_users_hit_ratio = cold_users_events_with_recs.groupby("user_id").agg(hits=("avg_rating", lambda x: (~x.isnull()).mean()))

print(f"Доля пользователей без релевантных рекомендаций: {(cold_users_hit_ratio == 0).mean().iat[0]:.2f}")
print(f"Среднее покрытие пользователей: {cold_users_hit_ratio[cold_users_hit_ratio != 0].mean().iat[0]:.2f}")

Доля пользователей без релевантных рекомендаций: 0.59
Среднее покрытие пользователей: 0.44


In [18]:
# Оцените степень разреженности U-I-матрицы, построенной на основе events. 

# Уникальные пользователи и товары
n_users = events['user_id'].nunique()
n_items = events['item_id'].nunique()

# Количество реальных оценок
n_ratings = len(events)

# Общее возможное взаимодействий
total_possible = n_users * n_items

# Степень разреженности (в %)
sparsity = (1 - n_ratings / total_possible) * 100

print(
    f"Уникальные пользователи: {n_users:,}\n"
    f"Уникальные товары: {n_items:,}\n"
    f"Количество оценок: {n_ratings:,}\n"
    f"Разреженность матрицы: {sparsity:.2f}%"
)

Уникальные пользователи: 430,585
Уникальные товары: 41,673
Количество оценок: 11,751,086
Разреженность матрицы: 99.93%


# === Базовые подходы: коллаборативная фильтрация

# === Базовые подходы: контентные рекомендации

# === Базовые подходы: валидация

# === Двухстадийный подход: метрики

# === Двухстадийный подход: модель

# === Двухстадийный подход: построение признаков